In [49]:
begin
	using Colors, ColorVectorSpace, ImageShow, FileIO, ImageIO
	using PlutoUI
	using HypertextLiteral
	using LinearAlgebra
	using ForwardDiff
	using LinearAlgebra
	using NonlinearSolve
	using StaticArrays
    using Symbolics, Plots, LaTeXStrings
	using ForwardDiff: jacobian
    using Interact
	

	Base.showable(::MIME"text/html", ::AbstractMatrix{<:Colorant}) = false
end

In [2]:
using CSV,DataFrames


In [10]:
f₁(x) = x^2

f₁ (generic function with 1 method)

In [11]:
ForwardDiff.derivative(f₁, 5)

10

In [7]:
begin
	f₅(v) = 5sin(v[1]*v[2]) + 2*v[2]/4v[3]
	f₅(x,y,z) = 5sin(x*y) + 2*y/4z
end

f₅ (generic function with 2 methods)

In [8]:
ForwardDiff.gradient(f₅,[1,2,3])

3-element Vector{Float64}:
 -4.161468365471424
 -1.9140675160690452
 -0.1111111111111111

In [4]:
begin
	 idy((x,y)) = [x,y]
	 lin1((x,y)) =  [ 2x + 3y, -5x+4x ]
	 scalex(α) = ((x,y),) -> (α*x, y)
	 scaley(α) = ((x,y),) -> (x,   α*y)
	 rot(θ) = ((x,y),) -> [cos(θ)*x + sin(θ)*y, -sin(θ)*x + cos(θ)*y]
	 shear(α) = ((x,y),) -> [x+α*y,y]
	 genlin(a,b,c,d) = ((x,y),) -> [ a*x + b*y ; c*x + d*y ]
end

genlin (generic function with 1 method)

In [12]:
# T⁻¹ = id
#  T⁻¹ = rotate(α)
T⁻¹ = shear(1)
#   T⁻¹ = lin(A) # uses the scrubbable 
#   T⁻¹ = shear(α) ∘ shear(-α)
 # T⁻¹ = nonlin_shear(α)  
 #   T⁻¹ =   inverse(nonlin_shear(α))
#    T⁻¹ =  nonlin_shear(-α)
#  T⁻¹ =  xy 
# T⁻¹ = warp(α)
# T⁻¹ = ((x,y),)-> (x+α*y^2,y+α*x^2) # may be non-invertible

# T⁻¹ = ((x,y),)-> (x,y^2)  
# T⁻¹  = flipy ∘ ((x,y),) ->  ( (β*x - α*y)/(β - y)  , -h*y/ (β - y)   )

#4 (generic function with 1 method)

In [13]:
begin
	function warp(α)
		((x,y),)  -> begin
			r = √(x^2+y^2)
			θ=α*r
			rot(θ)([x,y])
		end
	end
	
	rθ(x) = ( norm(x), atan(x[2],x[1])) # maybe vectors are more readable here?
	
	xy((r,θ)) = ( r*cos(θ), r*sin(θ))
end

xy (generic function with 1 method)

In [16]:
warp(1)([4,5])

2-element Vector{Float64}:
 4.569521661655642
 4.4854734180083895

In [17]:
begin	
	warp₂(α,x,y) = rot(α*√(x^2+y^2))
	warp₂(α) = ((x,y),) -> warp₂(α,x,y)([x,y])	
end

warp₂ (generic function with 2 methods)

In [18]:
ForwardDiff.jacobian( warp(3.0), [4,5] )

2×2 Matrix{Float64}:
   7.06684    8.0157
 -10.6677   -11.9586

In [25]:
#線形変換全部
begin
	 id((x, y)) = SA[x, y]
	
	 scalex(α) = ((x, y),) -> SA[α*x,  y]
	 scaley(α) = ((x, y),) -> SA[x,   α*y]
	 scale(α)  = ((x, y),) -> SA[α*x, α*y]
	
	 swap((x, y))  = SA[y, x]
	 flipy((x, y)) = SA[x, -y]
	
	 rotate(θ) = ((x, y),) -> SA[cos(θ)*x + sin(θ)*y, -sin(θ)*x + cos(θ)*y]
	 shear(α)  = ((x, y),) -> SA[x + α*y, y]
end

shear (generic function with 1 method)

In [26]:
begin
	lin(a, b, c, d) = ((x, y),) -> (a*x + b*y, c*x + d*y)
	
	lin(A) = v-> A * [v...]  # linear algebra version using matrix multiplication
end

lin (generic function with 2 methods)

In [27]:
begin
  translate(α,β)  = ((x, y),) -> SA[x+α, y+β]   # affine, but not linear
	
  nonlin_shear(α) = ((x, y),) -> SA[x, y + α*x^2]
	
  warp(α)    = ((x, y),) -> rotate(α*√(x^2+y^2))(SA[x, y])
  xy((r, θ)) = SA[ r*cos(θ), r*sin(θ) ]
  rθ(x)      = SA[norm(x), atan(x[2],x[1]) ] 
  
  # exponentialish =  ((x,y),) -> [log(x+1.2), log(y+1.2)]
  # merc = ((x,y),) ->  [ log(x^2+y^2)/2 , atan(y,x) ] # (reim(log(complex(y,x)) ))
end

rθ (generic function with 1 method)

In [23]:
begin
	function transform_xy_to_ij(img::AbstractMatrix, x::Float64, y::Float64)
	# convert coordinate system xy to ij 
	# center image, and use "white" when out of the boundary
		
		rows, cols = size(img)
		m = max(cols, rows)	
		
	    # function to take xy to ij
		xy_to_ij =  translate(rows/2, cols/2) ∘ swap ∘ flipy ∘ scale(m/2)
		
		# apply the function and "snap to grid"
		i, j = floor.(Int, xy_to_ij((x, y))) 
	
	end
	
	function getpixel(img,i::Int,j::Int; circular::Bool=false, r::Real=200)   
		#  grab image color or place default
		rows, cols = size(img)
		m = max(cols,rows)
		if circular
			c = (i-rows/2)^2 + (j-cols/2)^2 ≤ r*m^2/4
		else
			c = true
		end
		
		if 1 < i ≤ rows && 1 < j ≤ cols && c
			img[i, j]
		else
			# white(img[1, 1])
			black(img[1,1])
		end
		
	end
	
	
	# function getpixel(img,x::Float64,y::Float64)
	# 	i,j = transform_xy_to_ij(img,x,y)
	# 	getpixel(img,i,j)
	# end
	
	function transform_ij_to_xy(i::Int,j::Int,pixels)
	
	   ij_to_xy =  scale(2/pixels) ∘ flipy ∘ swap ∘ translate(-pixels/2,-pixels/2)
	   ij_to_xy([i,j])
	end

	    
end

transform_ij_to_xy (generic function with 1 method)

In [33]:
img = load(download("https://user-images.githubusercontent.com/6933510/110868198-713faa80-82c8-11eb-8264-d69df4509f49.png"));

In [66]:
begin
		[			    
			begin
			
			 x, y = transform_ij_to_xy(i,j, 500)
			
			X, Y = ( translate(-0.0,-0.0)  )([x,y])
			 X, Y = ( T⁻¹∘scale(1/15)∘translate(-0.0,-0.0) )([x,y])
			 i, j = transform_xy_to_ij(img,X,Y)
			 getpixel(img,i,j; circular=6.0, r=7.0)
			end	 
		
			for i = 1:500, j = 1:500
		]	
end

LoadError: TypeError: non-boolean (Float64) used in boolean context

In [65]:
transform_xy_to_ij(img,0.0,0.0)

2-element SVector{2, Int64} with indices SOneTo(2):
 321
 480

In [51]:
function standard_Newton(f, n, x_range, x0, ymin=-10, ymax=10)
    
    f′ = x -> ForwardDiff.derivative(f, x)


	p = plot(f, x_range, lw=3, ylim=(ymin, ymax), legend=:false, size=(400, 300))

	hline!([0.0], c="magenta", lw=3, ls=:dash)
	scatter!([x0], [0], c="green", ann=(x0, -5, L"x_0", 10))

	for i in 1:n

		plot!([x0, x0], [0, f(x0)], c=:gray, alpha=0.5)
		scatter!([x0], [f(x0)], c=:red)
		m = f′(x0)

		plot!(x_range, [straight(x0, f(x0), x, m) for x in x_range], 
			  c=:blue, alpha=0.5, ls=:dash, lw=2)

		x1 = x0 - f(x0) / m

		scatter!([x1], [0], c="green", ann=(x1, -5, L"x_%$i", 10))
		
		x0 = x1

	end

	p |> as_svg


end

standard_Newton (generic function with 3 methods)

In [63]:
let
	f(x) = x^2 - 2

	standard_Newton(f, 2, -1:0.01:10,2, -10, 70)
end

LoadError: UndefVarError: straight not defined

In [13]:
data=CSV.read("w2021.csv",DataFrame)

┌ Warning: thread = 1 warning: only found 1 / 3 columns around data row: 1. Filling remaining columns with `missing`
└ @ CSV /Users/aaaa/.julia/packages/CSV/1P1tQ/src/file.jl:576
┌ Warning: thread = 1 warning: only found 1 / 3 columns around data row: 2. Filling remaining columns with `missing`
└ @ CSV /Users/aaaa/.julia/packages/CSV/1P1tQ/src/file.jl:576
┌ Warning: thread = 1 warning: only found 1 / 3 columns around data row: 3. Filling remaining columns with `missing`
└ @ CSV /Users/aaaa/.julia/packages/CSV/1P1tQ/src/file.jl:576
┌ Warning: thread = 1 warning: only found 1 / 3 columns around data row: 4. Filling remaining columns with `missing`
└ @ CSV /Users/aaaa/.julia/packages/CSV/1P1tQ/src/file.jl:576
┌ Warning: thread = 1 warning: only found 1 / 3 columns around data row: 5. Filling remaining columns with `missing`
└ @ CSV /Users/aaaa/.julia/packages/CSV/1P1tQ/src/file.jl:576
┌ Warning: thread = 1 warning: only found 1 / 3 columns around data row: 6. Filling remaining columns wit

,�_�E�����[�h���������F2022/12/15 16
,String
1,",\x93\x8c\x8b\x9e,\x93\x8c\x8b\x9e,\x93\x8c\x8b\x9e,\x93\x8c\x8b\x9e,\x93\x8c\x8b\x9e,\x93\x8c\x8b\x9e,\x93\x8c\x8b\x9e,\x93\x8c\x8b\x9e,\x93\x8c\x8b\x9e,\x93\x8c\x8b\x9e,\x93\x8c\x8b\x9e,\x93\x8c\x8b\x9e,\x93\x8c\x8b\x9e,\x93\x8c\x8b\x9e,\x93\x8c\x8b\x9e,\x93\x8c\x8b\x9e"
2,"\x94N\x8c\x8e\x93\xfa,\x95\xbd\x8bϋC\x89\xb7(\x81\x8e),\x95\xbd\x8bϋC\x89\xb7(\x81\x8e),\x95\xbd\x8bϋC\x89\xb7(\x81\x8e),\x8dō\x82\x8bC\x89\xb7(\x81\x8e),\x8dō\x82\x8bC\x89\xb7(\x81\x8e),\x8dō\x82\x8bC\x89\xb7(\x81\x8e),\x8dŒ\xe1\x8bC\x89\xb7(\x81\x8e),\x8dŒ\xe1\x8bC\x89\xb7(\x81\x8e),\x8dŒ\xe1\x8bC\x89\xb7(\x81\x8e),\x8d\x87\x8cv\x91S\x93V\x93\xfa\x8e˗\xca(MJ/\x87u),\x8d\x87\x8cv\x91S\x93V\x93\xfa\x8e˗\xca(MJ/\x87u),\x8d\x87\x8cv\x91S\x93V\x93\xfa\x8e˗\xca(MJ/\x87u),\x93\xfa\x8fƎ\x9e\x8a\xd4(\x8e\x9e\x8a\xd4),\x93\xfa\x8fƎ\x9e\x8a\xd4(\x8e\x9e\x8a\xd4),\x93\xfa\x8fƎ\x9e\x8a\xd4(\x8e\x9e\x8a\xd4),\x93\xfa\x8fƎ\x9e\x8a\xd4(\x8e\x9e\x8a\xd4)"
3,",,,,,,,,,,,,,,,,"
4,",,\x95i\x8e\xbf\x8f\xee\x95\xf1,\x8bώ\xbf\x94ԍ\x86,,\x95i\x8e\xbf\x8f\xee\x95\xf1,\x8bώ\xbf\x94ԍ\x86,,\x95i\x8e\xbf\x8f\xee\x95\xf1,\x8bώ\xbf\x94ԍ\x86,,\x95i\x8e\xbf\x8f\xee\x95\xf1,\x8bώ\xbf\x94ԍ\x86,,\x8c\xbb\x8fۂȂ\xb5\x8f\xee\x95\xf1,\x95i\x8e\xbf\x8f\xee\x95\xf1,\x8bώ\xbf\x94ԍ\x86"
5,"2021/1/1,4.4,8,1,10.5,8,1,-1.3,8,1,11.72,8,1,8.7,0,8,1"
6,"2021/1/2,4.8,8,1,10.8,8,1,0.1,8,1,11.47,8,1,8.8,0,8,1"
7,"2021/1/3,3.7,8,1,8.4,8,1,-0.4,8,1,10.63,8,1,8.8,0,8,1"
8,"2021/1/4,5.8,8,1,11.4,8,1,1.3,8,1,10.27,8,1,7.7,0,8,1"
9,"2021/1/5,6.0,8,1,10.0,8,1,2.5,8,1,5.58,8,1,1.0,0,8,1"
